In [ ]:
#Libraries:

import pandas as pd
import numpy as np

from google.colab import files

##Data

In [ ]:
# Input data:

# coal mass flow[kg_coal/h]:
m_dot_coal=[120,140,182,136,651]
m_dot_coal=[float(x) for x in m_dot_coal]

# coal composition[%]:
coal_composition={"C":0.6561,"H":0.01215,"O":0.1375,"S":0.38*10**(-3),"H2O":0.1048, "Ash":0.774}

# Molar mass [kg/kmol]:
molar_mass={"C":12,"H":1,"O":16,"S":32,"H":1,"N":14}

In [ ]:
# Other molar mass [kg/kmol]:

molar_mass["H2O"]=2*molar_mass["H"]+molar_mass["O"]
molar_mass["N2"]=2*molar_mass["N"]
molar_mass["O2"]=2*molar_mass["O"]
molar_mass["CO2"]=2*molar_mass["O"]+molar_mass["C"]
molar_mass["SO2"]=2*molar_mass["O"]+molar_mass["S"]
molar_mass["Air"]=28.9

##CO2 aproximation:




In [ ]:
# reagent moles [kmol/kg_coal]
reagent_moles={"C":0.6561/molar_mass["C"],"H":0.01215/molar_mass["H"],"O":0.1375/molar_mass["O"],"S":0.38*10**(-3)/molar_mass["S"],"a_t":0.05647}

# Product moles [kmol/kg_coal]
product_moles={"CO2":0.05468,"H2O":0.02426,"SO2":1.188*10**(-5),"N2":0.2134}

Ecuación  estequimétrica de combustión:

$$C_{0.05468}H_{0.02426}O_{8.594\times10^{-3}}S_{1.188\times{10^{-5}}}+a_t(O_2+3.78N_2) \\ \to 0.05468CO_2+0.01215H_2O+1.188\times 10^{-5}SO_2+0.2134N_2 \ \left[\frac{kmol}{kg_{coal}} \right]$$


To calculate the $CO_2$ in the chimney fumes in boiler 1, proceed as follows:

$$\dot{m}_{CO_2,1}=0.05468\frac{kmol_{CO_2}}{kg_{coal}}\times\frac{44kg_{CO_2}}{1kmol_{CO_2}}\times\frac{120kg_{coal}}{1h}=288.684 \frac{kg_{CO_2}}{h}$$

The same procedure applies to all:

In [ ]:
# CO2 flow [kg/h]

boiler=["Térmica 1", "Térmica 2"," Térmica 3", "Térmica 4", "Térmica JCT"]
exhaust_gases=pd.DataFrame({ "Coal Flow rate [kg/h]":m_dot_coal})
exhaust_gases.index=boiler
exhaust_gases["CO2 Flow rate [kg/h]"]=product_moles["CO2"]*molar_mass["CO2"]*exhaust_gases["Coal Flow rate [kg/h]"]
exhaust_gases

,Coal Flow rate [kg/h],CO2 Flow rate [kg/h]
Térmica 1,120.0,288.71040
Térmica 2,140.0,336.82880
Térmica 3,182.0,437.87744
Térmica 4,136.0,327.20512
Térmica JCT,651.0,1566.25392


##Chimney gases approximation:

In [ ]:
product_moles_2={"CO2":0.05468,"H2O":0.01215,"SO2":1.188*10**(-5),"N2":0.3842,"O2":0.0452}

# Gases [kg gas/kg coal]
exhaust_gases_mass=dict(zip([x for x in list(product_moles_2.keys())],[product_moles_2[x]*molar_mass[x] for x in list(product_moles_2.keys())]))
exhaust_gases_mass["H2O"]=exhaust_gases_mass["H2O"]+1.8*reagent_moles['a_t']*molar_mass["Air"]*0.1/0.9+0.1048

exhaust_gases=pd.DataFrame({ "Coal Flow rate [kg/h]":m_dot_coal})
exhaust_gases.index=boiler
exhaust_gases["CO2 Flow rate [kg/h]"]=product_moles["CO2"]*molar_mass["CO2"]*exhaust_gases["Coal Flow rate [kg/h]"]
exhaust_gases_qty=exhaust_gases.copy()

for i in range(1,len(exhaust_gases_mass)):
  exhaust_gases_qty[list(exhaust_gases_mass.keys())[i]+" flow rate [kg/h]"]= exhaust_gases_mass[list(exhaust_gases_mass.keys())[i]]*exhaust_gases_qty["Coal Flow rate [kg/h]"]

exhaust_gases_qty.to_excel('ExhaustGasesQty.xlsx')
files.download('ExhaustGasesQty.xlsx')

exhaust_gas=sum(exhaust_gases_mass.values())
exhaust_gases['Chimney gases [kg/h]']=exhaust_gas*exhaust_gases["Coal Flow rate [kg/h]"]

exhaust_gases

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,Coal Flow rate [kg/h],CO2 Flow rate [kg/h],Chimney gases [kg/h]
Térmica 1,120.0,288.71040,1831.269230
Térmica 2,140.0,336.82880,2136.480769
Térmica 3,182.0,437.87744,2777.424999
Térmica 4,136.0,327.20512,2075.438461
Térmica JCT,651.0,1566.25392,9934.635575


## Volumetric flow aproximation (ideal gas)


In [ ]:
total_gases_moles= sum(list(product_moles_2.values()))-product_moles_2['H2O']+exhaust_gases_mass['H2O']*(1/molar_mass['H2O'])

# Molar fraction [kmol/kmol]
molar_fractions=dict(zip([x for x in list(product_moles_2.keys())],[product_moles_2[x]/total_gases_moles for x in list(product_moles_2.keys())]))
molar_fractions['H2O']=(exhaust_gases_mass['H2O']*(1/molar_mass['H2O']))/total_gases_moles
M_gases=sum([molar_fractions[x]*molar_mass[x] for x in list(molar_fractions.keys())])

#Universal ideal gas constant in kJ/kmol*K
R_u=8.315

#Volumetric Flow
exhaust_gases['Chimney gases Temp[K]']=[523.5,445,445,445,445]
exhaust_gases['Chimney gases Pressure[kPa]']=[79.74,79.74,79.74,79.74,79.74]
exhaust_gases['Chimney gases volumetric Flow [m3/h]']=[(exhaust_gases['Chimney gases [kg/h]'][x]*R_u*exhaust_gases['Chimney gases Temp[K]'][x])/(M_gases*exhaust_gases['Chimney gases Pressure[kPa]'][x]) for x in list(exhaust_gases.index)]

exhaust_gases.to_excel('ExhaustGases.xlsx')
files.download('ExhaustGases.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Volumetric flow real (based on june Isokinetic inform)[m3/h]
volumetric_flow_real=dict(zip([x for x in list(exhaust_gases.index)],list(3338.1*np.ones(5))))

#Error respect by june Isokinetic inform [%]
Volumetric_flow_error=dict(zip([x for x in list(exhaust_gases.index)],[np.abs(100*(exhaust_gases['Chimney gases volumetric Flow [m3/h]'][x]-volumetric_flow_real[x])/volumetric_flow_real[x]) for x in list(volumetric_flow_real.keys())]))


In [ ]:
Air_per_kgCoal=1.8*reagent_moles["a_t"]*molar_mass['Air']*4.76

In [ ]:
4+0.746*0.15

4.1119

In [ ]:
price =1047.23

In [ ]:
4.1119*1047.23

4306.105037

In [ ]:
=4306.105037*4

SyntaxError: invalid syntax (<ipython-input-4-01ab6f7a01c2>, line 1)

In [ ]:
4306.105037*4

17224.420148

In [ ]:
tot = 0
for i in [5, 4, 3, 2, 1] :
    tot = tot + 1
print(tot)

5


In [ ]:
friends = ['Joseph', 'Glenn', 'Sally']
for friend in friends :
     print('Happy New Year:',  friend)
print('Done!')

Happy New Year: Joseph
Happy New Year: Glenn
Happy New Year: Sally
Done!


In [ ]:
smallest_so_far = -1
for the_num in [9, 41, 12, 3, 74, 15] :
   if the_num < smallest_so_far :
      smallest_so_far = the_num
print(smallest_so_far)

-1


In [ ]:
x = 'From marquard@uct.ac.za'
print(x[8])


q


In [ ]:
print(x[15:18])

ct.


In [ ]:


print(x[14:17])

uct


In [ ]:
greet = 'Hello Bob'
print(greet.upper())

HELLO BOB


In [ ]:
'Hello Bob'.join()

TypeError: str.join() takes exactly one argument (0 given)

In [ ]:
data = 'From stephen.marquard@uct.ac.za Sat Jan  5 09:14:16 2008'
pos = data.find('.')
print(data[pos:pos+3])

.ma
